# Recipe Scaler Implementation Plan

## 1. Goal
Add a feature to the Brew Detail page (`app/brew/[id]/page.tsx`) that allows users to scale the recipe by adjusting:
- **Target Volume (Menge)**: e.g., default 20L -> user changes to 50L.
- **Brewhouse Efficiency (SHA - Sudhausausbeute)**: e.g., default 65% -> user changes to 72%.

## 2. Technical Feasibility & Logic

### Scaling Logic (Simplified)

**Volume Scaling:**
If volume changes from $V_{old}$ to $V_{new}$:
- **Malts/Fermentables**: Scale linearly. $Amount_{new} = Amount_{old} \times (V_{new} / V_{old})$
- **Hops**: Scale linearly for bitterness (IBU). $Amount_{new} = Amount_{old} \times (V_{new} / V_{old})$
- **Yeast**: Usually packaged (e.g., 1 pack for 20L). Scaling is linear (grams), but packs are discrete. We will scale the grams/cells required.
- **Water**: Needs scaling.
    - Mash Water (Hauptguss): Depends on grain weight (ratio). $W_{mash} = Grain_{weight} \times Ratio$.
    - Sparge Water (Nachguss): Everything left to reach boil volume. Complex, but linear approximation works for display.
- **Times (Boil, Mash Steps)**: Do **NOT** scale.

**Efficiency (SHA) Scaling:**
Only affects **Malts/Sugars**.
If SHA changes from $SHA_{old}$ to $SHA_{new}$:
- Higher efficiency means LESS grain needed for same gravity.
- $Malt_{new} = Malt_{old} \times (SHA_{old} / SHA_{new})$

**Combined Scaling Factor:**
For malts:
$Factor = (V_{new} / V_{old}) \times (SHA_{old} / SHA_{new})$

For hops/water:
$Factor = (V_{new} / V_{old})$

### Challenges / "Logic Errors" in simple user plan
1.  **Water Calculations**: Just scaling the water values (Hauptguss/Nachguss) linearly with volume is often wrong because "dead space" (loss in kettle/tun) is constant, not relative.
    *   *Solution*: For a simple viewer, linear scaling is acceptable as an estimate. We don't know the user's equipment profile anyway.
2.  **Boil Off Rate**: Similarly, boil-off is constant (L/hour), not percentage based. If you boil 100L vs 20L, you don't boil off 5x more.
    *   *Solution*: Again, without equipment profile, linear scaling of the *final volume* is the only safe assumption for a generic recipe viewer.
3.  **Hop Utilization**: Utilization varies with boil gravity and volume. Simply scaling hops linearly might slightly miss the IBU target if boil dynamics change drastically.
    *   *Solution*: Negligible for viewer. Linear scaling is standard.

## 3. Implementation Steps

### A. State Management
Add state to `BrewDetailPage`:
- `scaleVolume`: initialized from `brew.data.batch_size` (or default 20).
- `scaleEfficiency`: initialized from `brew.data.efficiency` (or default 65-70).
- `originalVolume`: constant from DB.
- `originalEfficiency`: constant from DB.

### B. UI Component (Scaler Controls)
Where? Top of the recipe data section ("Technische Details").
What? Two inputs (Number fields) with "Rezept skalieren" header. Or a "Skalieren" button that opens a small popover.
Let's put it inline above the specs grid.

### C. Display Logic
Create a helper function `scale(originalValue, type)`:
- If type == 'malt': apply $(V/V_0) * (E_0/E)$
- If type == 'other': apply $(V/V_0)$

Wrap displayed values (`MaltView`, `HopView`, etc.) to use this scaled value.

### D. Files to Modify
- `app/brew/[id]/page.tsx`: Main logic and state.
- Components inside that file (`MaltView`, `HopView`, etc.) need to accept the scaling factors or the scaled values.

## 4. Refinement
Since `MaltView` etc. are defined *outside* the main component currently, we have two options:
1. Move them inside (bad for performance/render).
2. Pass `scalingFactorMalt` and `scalingFactorVolume` as props to them. -> **Preferred**.

```tsx
const volumeFactor = scaleVolume / originalVolume;
const efficiencyFactor = originalEfficiency / scaleEfficiency;
const maltFactor = volumeFactor * efficiencyFactor;
```


In [ ]:
# Just placeholder to trigger my thoughts
# Plan:
# 1. Add `isSaved` state to `BrewDetailPage`.
# 2. Add `useEffect` that depends on `[id, userBreweries]`.
# 3. Query `brewery_saved_brews` to see if current brew is saved in ANY of the user's breweries.
# 4. Update UI button based on `isSaved`.


In [ ]:
# Helper for testing regex logic if needed

In [ ]:
pass

In [ ]:
# Hilfscode, um die Premium-Texte zu testen (kein direkter Einfluss auf App)
brewer_tier = "brewer"
bypass_limits = False

if not bypass_limits and brewer_tier == "brewer":
    print("Brewer Tier: AI ja, Limits nein.")
elif not bypass_limits:
    print("Free Tier: Limits gelten.")
else:
    print("Unlimited.")

In [1]:
def calculate_abv(og_plato, fg_plato):
    """
    Berechnet den Alkoholgehalt (Vol%) basierend auf Stammwürze (°P) und Restextrakt (°P).
    Verwendet eine Standard-Näherungsformel.
    """
    if og_plato is None or fg_plato is None:
        return 0.0
    
    # Näherungsformel : (Stammwürze - Restextrakt) / (2.0665 - 0.010665 * Stammwürze)
    if og_plato == 0: return 0
    return (og_plato - fg_plato) / (2.0665 - 0.010665 * og_plato)

# Test
print(f"12°P -> 3°P = {calculate_abv(12, 3):.2f}%")
print(f"16°P -> 4°P = {calculate_abv(16, 4):.2f}%")

12°P -> 3°P = 4.64%
16°P -> 4°P = 6.33%


In [2]:
import math

def calculate_ebc(batch_volume_liters, malts):
    """
    Calculates EBC color using Morey's formula adapted for metric units.
    malts: list of dicts { 'amount': kg, 'color_ebc': ebc }
    """
    if not batch_volume_liters or batch_volume_liters <= 0:
        return 0
    
    mcu_total = 0
    for m in malts:
        # Convert KG to Lbs (1kg = 2.20462 lbs)
        # Convert EBC to Lovibond (Lovibond = (EBC * 0.508) roughly or (EBC / 1.97)) -> Formula usually uses specific conversions
        # Actually, let's look at the MCU imperial definition: (Weight_lbs * Color_L) / Vol_gal
        
        weight_lbs = m['amount'] * 2.20462
        # EBC to Lovibond approximation: L = EBC / 1.97
        color_lovibond = m['color_ebc'] / 1.97
        vol_gal = batch_volume_liters * 0.264172
        
        mcu = (weight_lbs * color_lovibond) / vol_gal
        mcu_total += mcu
        
    # Morey Equation for SRM: SRM = 1.4922 * (MCU ^ 0.6859)
    srm = 1.4922 * (mcu_total ** 0.6859)
    
    # Convert SRM back to EBC: EBC = SRM * 1.97
    return srm * 1.97

def calculate_ibu(batch_volume_liters, boil_gravity, hops):
    """
    Calculates IBU using Tinseth formula.
    hops: list of dicts { 'amount': g, 'alpha': %, 'time': min, 'usage': 'Boil' }
    boil_gravity: SG (e.g., 1.050)
    """
    if not batch_volume_liters or batch_volume_liters <= 0:
        return 0
        
    total_ibu = 0
    
    for h in hops:
        if h.get('usage') != 'Boil' and h.get('usage') != 'First Wort': 
            continue # Skip dry hop / mash for bitterness contribution (simplified)
            
        amount_g = h['amount']
        alpha = h['alpha']
        time = h['time']
        
        # Tinseth Components
        
        # 1. Bigness Factor (Gravity effect)
        # 1.65 * 0.000125^(Gb - 1)
        bigness = 1.65 * (0.000125 ** (boil_gravity - 1))
        
        # 2. Boil Time Factor
        # (1 - e^(-0.04 * time)) / 4.15
        boil_factor = (1 - math.exp(-0.04 * time)) / 4.15
        
        utilization = bigness * boil_factor
        
        # IBUs = (Utilization * mg_alpha_acid) / Liters
        # mg_alpha = weight_g * alpha_percent * 10 
        # (e.g. 5% alpha in 10g hop -> 0.5g alpha = 500mg. 10g * 5 * 10 is wrong? 10g * 0.05 = 0.5g = 500mg. So it's g * (alpha/100) * 1000 = g * alpha * 10. Yes.)
        mg_alpha = amount_g * alpha * 10
        
        ibu = (utilization * mg_alpha) / batch_volume_liters
        total_ibu += ibu
        
    return total_ibu

# Test Data
malts_test = [
    {'amount': 4.5, 'color_ebc': 3}, # Pilsner
    {'amount': 0.5, 'color_ebc': 20} # Munich
]
hops_test = [
    {'amount': 30, 'alpha': 6.0, 'time': 60, 'usage': 'Boil'},
    {'amount': 20, 'alpha': 3.0, 'time': 10, 'usage': 'Boil'}
]
vol = 20
og = 1.050

print(f"Calculated EBC: {calculate_ebc(vol, malts_test):.1f}")
print(f"Calculated IBU: {calculate_ibu(vol, og, hops_test):.1f}")


Calculated EBC: 8.8
Calculated IBU: 23.3


In [ ]:
# Python prototyping for ABV and Carbonation
def calculate_abv(og, fg):
    return (og - fg) * 131.25

def calculate_priming_sugar(vol_co2_target, temp_c, beer_vol_l):
    # CO2 already dissolved at temp_c
    # Formula: CO2_in_beer = 3.0378 - (0.050062 * temp) + (0.00026555 * temp^2)
    # (Approximation or use Henry's Law table)
    # Standard formula: 1.5 - 2.5 vols CO2
    
    # Simple approx for dissolved CO2 (Palmer)
    dissolved_co2 = 3.0378 * (2.71828 ** (-0.004 * temp_c * 1.8 + 32)) # This looks wrong
    
    # Better approx:
    # 1.013 * (1 - 0.004 * T) ? No.
    
    # Let's use standard table approximation:
    # Dissolved CO2 (Volumes) = 3.0378 * e^(-0.0005 * T_F) ... no
    
    # Looking up widely used formula:
    # CO2_residual = 3.0378 - (0.050062 * T_F) + (0.00026555 * T_F^2)
    # where T_F is temp in Fahrenheit.
    
    t_f = (temp_c * 1.8) + 32
    co2_residual = 3.0378 - (0.050062 * t_f) + (0.00026555 * (t_f**2))
    
    if co2_residual < 0: co2_residual = 0
    
    co2_needed = vol_co2_target - co2_residual
    if co2_needed < 0: return 0
    
    # 1 vol CO2 = 1.96 g/L CO2 ?
    # Typically 4g/L sugar produces 0.23 vol CO2?
    
    # Standard: 4g suagar per liter raises CO2 by 1 volume? No.
    # Fermentation of 1g glucose produces 0.488g CO2.
    # Density of CO2 is roughly 1.96 g/L.
    # So 1g glucose -> 0.488g CO2 -> 0.25 L CO2 approx.
    # So 4g glucose -> 1 L CO2.
    
    # So roughly 4g/L sugar gives +1 Volume CO2.
    
    sugar_g_per_l = co2_needed * 4.0 # Approximation using household sugar (Sucrose)
    
    return sugar_g_per_l * beer_vol_l

print(calculate_abv(1.050, 1.010))
print(calculate_priming_sugar(2.5, 20, 20))